In [1]:
from pyspark.sql import DataFrame, Row
from functools import reduce
from pyspark.sql.types import *
from pyspark.sql.functions import *
from graphframes import *
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
import uuid

conf=SparkConf()
# conf.set("spark.executor.memory", "2g")
# conf.set("spark.driver.memory", "12g")
# conf.set("spark.cores.max", "6")

sc = SparkContext.getOrCreate(conf)

spark = SQLContext(sc)

23/12/13 13:31:48 WARN Utils: Your hostname, preacher resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface enp4s0)
23/12/13 13:31:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/13 13:31:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/13 13:31:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/home/preacher/anaconda3/envs/deltarun/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
matches_schema = StructType([
    StructField("pair_id", StringType(), True),
    StructField("base_sku_uuid", StringType(), True),
    StructField("base_source_store", StringType(), True),
    StructField("comp_sku_uuid", StringType(), True),
    StructField("comp_source_store", StringType(), True),
    StructField("seller_type", StringType(), True),
])

mdw_schema = StructType([
    StructField("pair_id", StringType(), True),
    StructField("base_sku_uuid", StringType(), True),
    StructField("base_source_store", StringType(), True),
    StructField("comp_sku_uuid", StringType(), True),
    StructField("comp_source_store", StringType(), True),
    StructField("seller_type", StringType(), True),
    StructField("bungee_audit_status", StringType(), True)
])
matche_data = [
    ["new1<>203359001<>homedepot<>homedepot_compNoSugg1p1<>g4103325<>bcg<>bcg","new1<>203359001<>homedepot<>homedepot","homedepot<>homedepot","compNoSugg1p1<>g4103325<>bcg<>bcg", "bcg<>bcg","1P"],
    ["new2<>100341666<>homedepot<>homedepot_compNoSugg1p2<>109589<>bcg<>bcg","new2<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compNoSugg1p2<>109589<>bcg<>bcg", "bcg<>bcg","1P"],
    
    [ "1<>203370169<>homedepot<>homedepot_compYes1pExactSugg1p<>967446<>bcg<>bcg","1<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compYes1pExactSugg1p<>967446<>bcg<>bcg","bcg<>bcg","1P"],
    [ "2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg","2<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compYes1pSimSugg1p<>967493<>bcg<>bcg","bcg<>bcg","1P"],
    [ "3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg","3<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compYes3pSimSugg3p<>109589<>bcg<>bcg","bcg<>bcg","3P"],
    [ "4<>53409<>homedepot<>homedepot_compYes3pExactSugg3p<>147664<>bcg<>bcg","4<>53409<>homedepot<>homedepot","homedepot<>homedepot","compYes3pExactSugg3p<>147664<>bcg<>bcg", "bcg<>bcg","3P"],
    
    ["new3<>53409<>homedepot<>homedepot_compNoSugg3p1<>147664<>bcg<>bcg","new3<>53409<>homedepot<>homedepot","homedepot<>homedepot","compNoSugg3p1<>147664<>bcg<>bcg", "bcg<>bcg","3P"],
    ["new4<>100341666<>homedepot<>homedepot_compNoSugg3p2<>109589<>bcg<>bcg","new4<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compNoSugg3p2<>109589<>bcg<>bcg", "bcg<>bcg","3P"],

    ["1<>203370169<>homedepot<>homedepot_compNo1pExactSugg1p<>967446<>bcg<>bcg","1<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pExactSugg1p<>967446<>bcg<>bcg","bcg<>bcg","1P"],
    ["2<>100670057<>homedepot<>homedepot_compNo1pSimSugg1p<>967493<>bcg<>bcg","2<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pSimSugg1p<>967493<>bcg<>bcg","bcg<>bcg","1P"],
    ["3<>100341666<>homedepot<>homedepot_compNo3pSimSugg3p<>109589<>bcg<>bcg","3<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compNo3pSimSugg3p<>109589<>bcg<>bcg","bcg<>bcg","3P"],
    ["4<>53409<>homedepot<>homedepot_compNo3pExactSugg3p<>147664<>bcg<>bcg","4<>53409<>homedepot<>homedepot","homedepot<>homedepot","compNo3pExactSugg3p<>147664<>bcg<>bcg", "bcg<>bcg","3P"],

    [ "1<>203370169<>homedepot<>homedepot_compNo1pExactSugg3p<>967446<>bcg<>bcg","1<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pExactSugg3p<>967446<>bcg<>bcg","bcg<>bcg","3P"],
    [ "2<>100670057<>homedepot<>homedepot_compNo1pSimSugg3p<>967493<>bcg<>bcg","2<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pSimSugg3p<>967493<>bcg<>bcg","bcg<>bcg","3P"],
    [ "3<>100341666<>homedepot<>homedepot_compNo3pSimSugg1p<>109589<>bcg<>bcg","3<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compNo3pSimSugg1p<>109589<>bcg<>bcg","bcg<>bcg","1P"],
    [ "4<>53409<>homedepot<>homedepot_compNo3pExactSugg1p<>147664<>bcg<>bcg","4<>53409<>homedepot<>homedepot","homedepot<>homedepot","compNo3pExactSugg1p<>147664<>bcg<>bcg", "bcg<>bcg","1P"],
    
    [ "5<>203370169<>homedepot<>homedepot_compNo1pExactSugg1p<>967446<>bcg<>bcg","5<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pExactSugg1p<>967446<>bcg<>bcg","bcg<>bcg","1P"],
    [ "5<>203370169<>homedepot<>homedepot_compNo3pExactSugg3p<>147664<>bcg<>bcg","5<>203370169<>homedepot<>homedepot","homedepot<>homedepot","compNo3pExactSugg3p<>147664<>bcg<>bcg", "bcg<>bcg","3P"],
    [ "5<>203370169<>homedepot<>homedepot_compNo1pExactSugg3p<>967446<>bcg<>bcg","5<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pExactSugg3p<>967446<>bcg<>bcg","bcg<>bcg","3P"],
    [ "5<>203370169<>homedepot<>homedepot_compNo3pExactSugg1p<>147664<>bcg<>bcg","5<>203370169<>homedepot<>homedepot","homedepot<>homedepot","compNo3pExactSugg1p<>147664<>bcg<>bcg", "bcg<>bcg","1P"],
    
    [ "6<>100670057<>homedepot<>homedepot_compNo1pSimSugg1p<>967493<>bcg<>bcg","6<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pSimSugg1p<>967493<>bcg<>bcg","bcg<>bcg","1P"],
    [ "6<>100670057<>homedepot<>homedepot_compNo3pSimSugg3p<>967493<>bcg<>bcg","6<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compNo3pSimSugg3p<>967493<>bcg<>bcg","bcg<>bcg","3P"],
    [ "6<>100670057<>homedepot<>homedepot_compNo1pSimSugg3p<>967493<>bcg<>bcg","6<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compNo1pSimSugg3p<>967493<>bcg<>bcg","bcg<>bcg","3P"],
    [ "6<>100670057<>homedepot<>homedepot_compNo3pSimSugg1p<>967493<>bcg<>bcg","6<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compNo3pSimSugg1p<>967493<>bcg<>bcg","bcg<>bcg","1P"],

    [ "1<>203370169<>homedepot<>homedepot_baseSrcStrYesCompSrcStrNo1<>967446<>amazon<>amazon","1<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","baseSrcStrYesCompSrcStrNo1<>967446<>amazon<>amazon","amazon<>amazon","1P", ],
    [ "2<>100670057<>homedepot<>homedepot_baseSrcStrYesCompSrcStrNo2<>967493<>amazon<>amazon","2<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","baseSrcStrYesCompSrcStrNo2<>967493<>amazon<>amazon","amazon<>amazon","1P" ],
    [ "3<>100341666<>homedepot<>homedepot_baseSrcStrYesCompSrcStrNo3<>109589<>amazon<>amazon","3<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","baseSrcStrYesCompSrcStrNo3<>109589<>amazon<>amazon","amazon<>amazon","3P" ],
    [ "4<>53409<>homedepot<>homedepot_baseSrcStrYesCompSrcStrNo4<>147664<>amazon<>amazon","4<>53409<>homedepot<>homedepot","homedepot<>homedepot","baseSrcStrYesCompSrcStrNo3<>147664<>amazon<>amazon", "amazon<>amazon","3P" ],
    
    [ "5<>203370169<>homedepot<>homedepot_baseSrcStrYesCompSrcStrNo5<>967446<>amazon<>amazon","5<>203370169<>homedepot<>homedepot","homedepot<>homedepot","baseSrcStrYesCompSrcStrNo5<>967446<>amazon<>amazon","amazon<>amazon","1P" ],
    [ "5<>203370169<>homedepot<>homedepot_baseSrcStrYesCompSrcStrNo6<>147664<>amazon<>amazon","5<>203370169<>homedepot<>homedepot","homedepot<>homedepot","baseSrcStrYesCompSrcStrNo6<>147664<>amazon<>amazon", "amazon<>amazon","3P" ],

    [ "6<>100670057<>homedepot<>homedepot_baseSrcStrYesCompSrcStrNo7<>967493<>amazon<>amazon","6<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","baseSrcStrYesCompSrcStrNo7<>967493<>amazon<>amazon","amazon<>amazon","1P"],
    [ "6<>100670057<>homedepot<>homedepot_baseSrcStrYesCompSrcStrNo8<>967493<>amazon<>amazon","6<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","baseSrcStrYesCompSrcStrNo8<>967493<>amazon<>amazon","amazon<>amazon","3P"],

]
matches = spark.createDataFrame(matche_data, schema=matches_schema)

mdw_data = [
    [ "1<>203370169<>homedepot<>homedepot_compYes1pExactSugg1p<>967446<>bcg<>bcg","1<>203370169<>homedepot<>homedepot", "homedepot<>homedepot","compYes1pExactSugg1p<>967446<>bcg<>bcg","bcg<>bcg","1P","exact"],
    [ "2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg","2<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compYes1pSimSugg1p<>967493<>bcg<>bcg","bcg<>bcg","1P","similar"],
    [ "3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg","3<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","compYes3pSimSugg3p<>109589<>bcg<>bcg","bcg<>bcg","3P","similar"],
    [ "4<>53409<>homedepot<>homedepot_compYes3pExactSugg3p<>147664<>bcg<>bcg","4<>53409<>homedepot<>homedepot","homedepot<>homedepot","compYes3pExactSugg3p<>147664<>bcg<>bcg", "bcg<>bcg","3P","exact"],
    
    [ "5<>203370169<>homedepot<>homedepot_compYes1p3pExactSugg1p<>967446<>bcg<>bcg","5<>203370169<>homedepot<>homedepot","homedepot<>homedepot","compYes1p3pExactSugg1p<>967446<>bcg<>bcg","bcg<>bcg","1P","exact"],
    [ "5<>203370169<>homedepot<>homedepot_compYes3p3pExactSugg3p<>147664<>bcg<>bcg","5<>203370169<>homedepot<>homedepot","homedepot<>homedepot","compYes3p3pExactSugg3p<>147664<>bcg<>bcg", "bcg<>bcg","3P","exact"],

    [ "6<>100670057<>homedepot<>homedepot_compYes1p3pSimSugg1p<>967493<>bcg<>bcg","6<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compYes1p3pSimSugg1p<>967493<>bcg<>bcg","bcg<>bcg","1P","similar"],
    [ "6<>100670057<>homedepot<>homedepot_compYes1p3pSimSugg3p<>967493<>bcg<>bcg","6<>100670057<>homedepot<>homedepot", "homedepot<>homedepot","compYes1p3pSimSugg3p<>967493<>bcg<>bcg","bcg<>bcg","3P","similar"],
]
mdw = spark.createDataFrame(mdw_data, schema=mdw_schema)

In [3]:
matches.show()
mdw.show(truncate= False)

+--------------------+--------------------+--------------------+--------------------+-----------------+-----------+
|             pair_id|       base_sku_uuid|   base_source_store|       comp_sku_uuid|comp_source_store|seller_type|
+--------------------+--------------------+--------------------+--------------------+-----------------+-----------+
|new1<>203359001<>...|new1<>203359001<>...|homedepot<>homedepot|compNoSugg1p1<>g4...|         bcg<>bcg|         1P|
|new2<>100341666<>...|new2<>100341666<>...|homedepot<>homedepot|compNoSugg1p2<>10...|         bcg<>bcg|         1P|
|1<>203370169<>hom...|1<>203370169<>hom...|homedepot<>homedepot|compYes1pExactSug...|         bcg<>bcg|         1P|
|2<>100670057<>hom...|2<>100670057<>hom...|homedepot<>homedepot|compYes1pSimSugg1...|         bcg<>bcg|         1P|
|3<>100341666<>hom...|3<>100341666<>hom...|homedepot<>homedepot|compYes3pSimSugg3...|         bcg<>bcg|         3P|
|4<>53409<>homedep...|4<>53409<>homedep...|homedepot<>homedepot|compYes3

In [4]:
def combine_dfs(df_list: list[DataFrame]) -> DataFrame:
    print("Combining dfs")
    if df_list is None:
        return None
    
    df_list = [df for df in df_list if df is not None]
    if len(df_list) == 0:
        return None
    if len(df_list) == 1:
        return df_list[0]
    combined_df = reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), df_list)
    return combined_df

In [5]:
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql import DataFrame
from bungee_utils.spark_utils.function.dataframe_util import *
from pyspark.sql.window import Window

class Match_Pruner:
    def __init__(self, mdw: DataFrame, match_suggestion: DataFrame, config: dict) -> None:
        self.mdw = mdw
        self.match_suggestion = match_suggestion
        self.config = config

    def prune_matches(self):
        self.mdw, match_suggestion = self.preprocessing_matches(self.mdw, self.match_suggestion)
        similar_matches, exact_matches, unaudited_matches = self.separate_audited_and_unaudited_matches(match_suggestion, self.mdw)
        similar_matches.show(truncate = False)
        exact_matches.show(truncate = False)
        unaudited_matches.show(truncate = False)
        pruned_suggestion, match_suggestion = self.prune_match_based_on_client_config(similar_matches, exact_matches, unaudited_matches)
        updated_match_suggestion = combine_dfs([pruned_suggestion, match_suggestion]).drop("audited_match_seller_type_list","audited_match_seller_type")
        updated_match_suggestion.cache()
        if updated_match_suggestion.count() == unaudited_matches.count():
            print("matches are not missing")
        elif updated_match_suggestion.count() < unaudited_matches.count(): 
            print("matches are missing")
        else :
            print("duplicate matches are present")
        if pruned_suggestion is not None:
            pruned_suggestion.show(truncate = False)
        if match_suggestion is not None:
            match_suggestion.show(truncate = False)
        return updated_match_suggestion

    def preprocessing_matches(self, mdw: DataFrame, match_suggestion: DataFrame):
        match_suggestion = match_suggestion.withColumn("seller_type", lower(col("seller_type")))
        mdw = mdw.withColumn("seller_type", lower(col("seller_type")))\
            .withColumn("bungee_audit_status", when( col("bungee_audit_status").isNotNull(), lower(col("bungee_audit_status")) ).otherwise(col("bungee_audit_status")) )

        return mdw, match_suggestion
    
    def separate_audited_and_unaudited_matches(self, match_suggestion:DataFrame, mdw:DataFrame):
        print("separate_audited_and_unaudited_matches")
        window_spec = Window.partitionBy("base_sku_uuid")
        similar_matches = mdw.filter(col('bungee_audit_status').isin(['equivalent', 'similar']))
        similar_matches = similar_matches.withColumn("audited_match_seller_type_list", concat_ws( "_", array_sort( collect_set(col("seller_type")).over(window_spec) ) ) )
        similar_matches = similar_matches.withColumn("misc_info", concat(lit("similar_match_"), col("pair_id")) ).\
                        select("base_sku_uuid", "audited_match_seller_type_list", "misc_info", col("seller_type").alias("audited_match_seller_type"))
               
        exact_matches = mdw.filter(col('bungee_audit_status').isin(['exact']))
        exact_matches = exact_matches.withColumn("audited_match_seller_type_list", concat_ws("_", array_sort( collect_set(col("seller_type")).over(window_spec) ) ) )
        exact_matches = exact_matches.withColumn("misc_info", concat(lit("exact_match_"), col("pair_id")) ).\
                        select("base_sku_uuid", "audited_match_seller_type_list", "misc_info", col("seller_type").alias("audited_match_seller_type"))
        unaudited_matches = match_suggestion.join(mdw.select("pair_id"), 'pair_id', 'left_anti')
        
        return similar_matches, exact_matches, unaudited_matches

    def prune_match_based_on_client_config(self, similar_matches:DataFrame , exact_matches:DataFrame , audited_base_product_unaudited_matches:DataFrame ):
        print("Match Pruning Started")
        pruned_suggestion = None
        match_suggestion = None
        if self.config['match_type'] == 'exact' or self.config['match_type'] == 'exact_over_similar':
            pruned_suggestion, match_suggestion = self.prune_for_exact_match_config(exact_matches, audited_base_product_unaudited_matches) 

        elif self.config['match_type'] == 'similar' or self.config['match_type'] == 'exact_or_similar':
            exact_and_similar_match = exact_matches.union(similar_matches)
            pruned_suggestion, match_suggestion = self.prune_for_exact_similar_match(audited_base_product_unaudited_matches, exact_and_similar_match)
            
        if pruned_suggestion is not None:
            pruned_suggestion = pruned_suggestion.withColumn("bungee_audit_status", lit("PRUNED"))
            pruned_suggestion = pruned_suggestion.withColumn("bungee_auditor", lit("match_management_system"))
            pruned_suggestion = pruned_suggestion.withColumn("updated_by", lit("match_management_system"))
            pruned_suggestion = pruned_suggestion.withColumn("updated_date", current_timestamp())
            
        if match_suggestion is not None:
            match_suggestion = match_suggestion.withColumn("bungee_audit_status", lit("UNAUDITED"))
            match_suggestion = match_suggestion.withColumn("bungee_auditor", lit(None))
            match_suggestion = match_suggestion.withColumn("updated_by", lit(None))
            match_suggestion = match_suggestion.withColumn("updated_date", lit(None))
            match_suggestion = match_suggestion.withColumn("misc_info", lit(None))
            
        return pruned_suggestion, match_suggestion

    def prune_for_exact_similar_match(self, audited_base_product_unaudited_matches: DataFrame, exact_and_similar_match: DataFrame):
        print("Pruning Exact Sim Matches")
        if self.config['seller_type'] == '1p':
            audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_and_similar_match.filter(col("audited_match_seller_type") == '1p'), "base_sku_uuid", "left")
            audited_base_product_unaudited_matches.show(truncate =False)
            if self.config['cardinality'] == '1':
                pruned_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNotNull() | (col("seller_type") == '3p')).\
                                    withColumn("misc_info", when( col("seller_type") == '3p', lit("seller_type_config_1p")).otherwise(col("misc_info")) )
                match_suggestion = audited_base_product_unaudited_matches.filter( col("misc_info").isNull() & (col("seller_type") == '1p') )
            if self.config['cardinality'] == 'n':
                pruned_suggestion = audited_base_product_unaudited_matches.filter( (col("seller_type") == '3p')).\
                                    withColumn("misc_info", when( col("seller_type") == '3p', lit("seller_type_config_3p")).otherwise(col("misc_info")) )
                match_suggestion = audited_base_product_unaudited_matches.filter( (col("seller_type") == '1p'))
        elif self.config['seller_type'] == '3p':
            audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_and_similar_match.filter(col("audited_match_seller_type") == '3p'), "base_sku_uuid", "left")
            if self.config['cardinality'] == '1':
                pruned_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNotNull() | (col("seller_type") == '1p')).\
                                    withColumn("misc_info", when( col("seller_type") == '1p', lit("seller_type_config_1p")).otherwise(col("misc_info")) )
                match_suggestion = audited_base_product_unaudited_matches.filter( col("misc_info").isNull() & (col("seller_type") == '3p') )
            if self.config['cardinality'] == 'n':
                pruned_suggestion = audited_base_product_unaudited_matches.filter( (col("seller_type") == '1p')).\
                                    withColumn("misc_info", when( col("seller_type") == '1p', lit("seller_type_config_3p")).otherwise(col("misc_info")) )
                match_suggestion = audited_base_product_unaudited_matches.filter( (col("seller_type") == '3p'))
        elif self.config['seller_type'] == '1p_or_3p':
            exact_and_similar_match = exact_and_similar_match.withColumn("audited_match_seller_type_list", when( col("audited_match_seller_type_list") == "1p_3p", "1p").otherwise(col("audited_match_seller_type_list")) ).\
                                            filter( col("audited_match_seller_type_list") == col("audited_match_seller_type"))
            audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_and_similar_match, "base_sku_uuid", "left")
            if self.config['cardinality'] == '1':
                pruned_suggestion = audited_base_product_unaudited_matches.filter(col("audited_match_seller_type_list").isNotNull() )
                match_suggestion = audited_base_product_unaudited_matches.filter(col("audited_match_seller_type_list").isNull() )
            if self.config['cardinality'] == 'n':
                pruned_suggestion = None
                match_suggestion = audited_base_product_unaudited_matches
        elif self.config['seller_type'] == '1p_over_3p':
            exact_and_similar_match = exact_and_similar_match.withColumn("audited_match_seller_type_list", when( col("audited_match_seller_type_list") == "1p_3p", "1p").otherwise(col("audited_match_seller_type_list")) ).\
                                            filter( col("audited_match_seller_type_list") == col("audited_match_seller_type"))
            audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_and_similar_match, "base_sku_uuid", "left")
            audited_base_product_unaudited_matches.printSchema()
            if self.config['cardinality'] == '1':
                pruned_suggestion = audited_base_product_unaudited_matches.filter( ~(col("audited_match_seller_type_list").isNull() | ((col("audited_match_seller_type_list") == "3p") & ( col('seller_type') == '1p' )) ) )
                match_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNull() | ( (col("audited_match_seller_type_list") == "3p") & (col('seller_type') == '1p') )  )
            if self.config['cardinality'] == 'n':
                pruned_suggestion = None
                match_suggestion = audited_base_product_unaudited_matches
        return pruned_suggestion,match_suggestion

    def prune_for_exact_match_config(self, exact_matches: DataFrame, audited_base_product_unaudited_matches: DataFrame):
        print("Pruning Exact Matches")
        if self.config['seller_type'] == '1p':
            audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_matches.filter(col("audited_match_seller_type") == '1p'), "base_sku_uuid", "left")
            pruned_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNotNull() | (col("seller_type") == '3p')).\
                                    withColumn("misc_info", when( col("seller_type") == '3p', lit("seller_type_config_1p")).otherwise(col("misc_info")) )
            match_suggestion = audited_base_product_unaudited_matches.filter( col("misc_info").isNull() & (col("seller_type") == '1p') )
            
        elif self.config['seller_type'] == '3p':
            audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_matches.filter(col("audited_match_seller_type") == '3p'), "base_sku_uuid", "left")
            if self.config['cardinality'] == '1':
                pruned_suggestion = audited_base_product_unaudited_matches.filter(col("audited_match_seller_type_list").isNotNull() | (col("seller_type") == '1p')).\
                                    withColumn("misc_info", when( col("seller_type") == '1p', lit("seller_type_config_3p")).otherwise(col("misc_info")) )
                match_suggestion = audited_base_product_unaudited_matches.filter(col("misc_info").isNull() & (col("seller_type") == '3p'))
            if self.config['cardinality'] == 'n':
                pruned_suggestion = audited_base_product_unaudited_matches.filter(col("seller_type") == '1p').withColumn("misc_info", lit("seller_type_config_3p"))
                match_suggestion = audited_base_product_unaudited_matches.filter(col("seller_type") == '3p')
                
        elif self.config['seller_type'] == '1p_or_3p':
            exact_matches = exact_matches.withColumn("audited_match_seller_type_list", when( col("audited_match_seller_type_list") == "1p_3p", "1p").otherwise(col("audited_match_seller_type_list")) ).\
                                            filter( col("audited_match_seller_type_list") == col("audited_match_seller_type"))
            audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_matches, "base_sku_uuid", "left")
            if self.config['cardinality'] == '1':
                pruned_suggestion = audited_base_product_unaudited_matches.filter(col("audited_match_seller_type_list").isNotNull() )
                match_suggestion = audited_base_product_unaudited_matches.filter(col("audited_match_seller_type_list").isNull() )
            if self.config['cardinality'] == 'n':
                pruned_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNotNull() & (col("seller_type") == "1p" )  & (col("audited_match_seller_type") == "1p" )) 
                match_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNull() | (col("seller_type") != "1p" )  | (col("audited_match_seller_type") != "1p" ) )
                
        elif self.config['seller_type'] == '1p_over_3p':
            exact_matches = exact_matches.withColumn("audited_match_seller_type_list", when( col("audited_match_seller_type_list") == "1p_3p", "1p").otherwise(col("audited_match_seller_type_list")) ).\
                                            filter( col("audited_match_seller_type_list") == col("audited_match_seller_type"))
            audited_base_product_unaudited_matches = audited_base_product_unaudited_matches.join(exact_matches, "base_sku_uuid", "left")
            if self.config['cardinality'] == '1':
                pruned_suggestion = audited_base_product_unaudited_matches.filter( ~(col("audited_match_seller_type_list").isNull() | ((col("audited_match_seller_type_list") == "3p") & ( col('seller_type') == '1p' )) ) )
                match_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNull() | ( (col("audited_match_seller_type_list") == "3p") & (col('seller_type') == '1p') )  )
            if self.config['cardinality'] == 'n':
                pruned_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNotNull() & (col("seller_type") == "1p" )  & (col("audited_match_seller_type") == "1p" )) 
                match_suggestion = audited_base_product_unaudited_matches.filter( col("audited_match_seller_type_list").isNull() | (col("seller_type") != "1p" )  | (col("audited_match_seller_type") != "1p" ) )
        return pruned_suggestion, match_suggestion
  
    
        
# INPUT
#   what we have aggregated in previous step (list of all match suggestion present with us)
#   complete match data warehouse (list of all match suggestion present in mdw)

# PROCESSING
   

In [7]:
config = {
    "seller_type" : "1p",
    "cardinality" : "n",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
updated_match_suggestion = pruner.prune_matches()
updated_match_suggestion.show(truncate=False, n = 100)


separate_audited_and_unaudited_matches
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|2<>100670057<>homedepot<>homedepot|1p                            |similar_match_2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg  |1p                       |
|3<>100341666<>homedepot<>homedepot|3p                            |similar_match_3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg  |3p                       |
|6<>100670057<>homedepot<>homedepot|1p_3p          

matches are not missing
+-------------------------------------+-------------------------------------------------------------------------------------+--------------------+--------------------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+-------------------+-----------------------+-----------------------+--------------------------+
|base_sku_uuid                        |pair_id                                                                              |base_source_store   |comp_sku_uuid                                     |comp_source_store|seller_type|audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|bungee_audit_status|bungee_auditor         |updated_by             |updated_date              |
+-------------------------------------+-

In [10]:
config = {
    "seller_type" : "1p",
    "cardinality" : "1",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
updated_match_suggestion = pruner.prune_matches()
updated_match_suggestion.show(truncate=False, n = 100)

separate_audited_and_unaudited_matches
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|2<>100670057<>homedepot<>homedepot|1p                            |similar_match_2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg  |1p                       |
|3<>100341666<>homedepot<>homedepot|3p                            |similar_match_3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg  |3p                       |
|6<>100670057<>homedepot<>homedepot|1p_3p          

23/12/13 12:36:40 WARN CacheManager: Asked to cache already cached data.


matches are not missing
+-------------------------------------+-------------------------------------------------------------------------------------+--------------------+--------------------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+-------------------+-----------------------+-----------------------+--------------------------+
|base_sku_uuid                        |pair_id                                                                              |base_source_store   |comp_sku_uuid                                     |comp_source_store|seller_type|audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|bungee_audit_status|bungee_auditor         |updated_by             |updated_date              |
+-------------------------------------+-

In [9]:
config = {
    "seller_type" : "3p",
    "cardinality" : "1",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
updated_match_suggestion = pruner.prune_matches()
updated_match_suggestion.show(truncate=False, n = 100)

n = 1
matche_data = [
    ["0000346318<>100341666<>homedepot<>homedepot_0003463186541<>109589<>bcg<>bcg","0000346318<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","0003463186541<>109589<>bcg<>bcg", "bcg<>bcg","3P"],
    ["00003463186541<>100341666<>homedepot<>homedepot_00034631<>109589<>bcg<>bcg","00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","00034631<>109589<>bcg<>bcg","bcg<>bcg","3P"],
    ["0000000001<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy","0000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy", "chewy<>chewy","3P"],
]

separate_audited_and_unaudited_matches
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|2<>100670057<>homedepot<>homedepot|1p                            |similar_match_2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg  |1p                       |
|3<>100341666<>homedepot<>homedepot|3p                            |similar_match_3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg  |3p                       |
|6<>100670057<>homedepot<>homedepot|1p_3p          

In [11]:
config = {
    "seller_type" : "3p",
    "cardinality" : "n",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
updated_match_suggestion = pruner.prune_matches()
updated_match_suggestion.show(truncate=False, n = 100)


separate_audited_and_unaudited_matches
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|2<>100670057<>homedepot<>homedepot|1p                            |similar_match_2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg  |1p                       |
|3<>100341666<>homedepot<>homedepot|3p                            |similar_match_3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg  |3p                       |
|6<>100670057<>homedepot<>homedepot|1p_3p          

In [12]:
config = {
    "seller_type" : "1p_or_3p",
    "cardinality" : "1",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
updated_match_suggestion = pruner.prune_matches()
updated_match_suggestion.show(truncate=False, n = 100)


matche_data = [
    ["00003463186541<>100341666<>homedepot<>homedepot_0003463186541<>109589<>bcg<>bcg","00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","0003463186541<>109589<>bcg<>bcg", "bcg<>bcg","3P"],
    ["00003463186541<>100341666<>homedepot<>homedepot_00034631<>109589<>bcg<>bcg","00003463186541<>100341666<>homedepot<>homedepot", "homedepot<>homedepot","00034631<>109589<>bcg<>bcg","bcg<>bcg","3P"],
    ["0000000000001<>53409<>petcarerx<>petcarerx_00000000<>147664<>chewy<>chewy","0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","00000000<>147664<>chewy<>chewy", "chewy<>chewy","3P"],
    ["0000000000001<>53409<>petcarerx<>petcarerx_0000<>147664<>chewy<>chewy","0000000000001<>53409<>petcarerx<>petcarerx","petcarerx<>petcarerx","0000<>147664<>chewy<>chewy", "chewy<>chewy","3P"],

]

separate_audited_and_unaudited_matches
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|2<>100670057<>homedepot<>homedepot|1p                            |similar_match_2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg  |1p                       |
|3<>100341666<>homedepot<>homedepot|3p                            |similar_match_3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg  |3p                       |
|6<>100670057<>homedepot<>homedepot|1p_3p          

matches are not missing
+----------------------------------+-------------------------------------------------------------------------------------+--------------------+--------------------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+-------------------+-----------------------+-----------------------+--------------------------+
|base_sku_uuid                     |pair_id                                                                              |base_source_store   |comp_sku_uuid                                     |comp_source_store|seller_type|audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|bungee_audit_status|bungee_auditor         |updated_by             |updated_date              |
+----------------------------------+----------

In [13]:
config = {
    "seller_type" : "1p_or_3p",
    "cardinality" : "n",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
updated_match_suggestion = pruner.prune_matches()
updated_match_suggestion.show(truncate=False, n = 100)


separate_audited_and_unaudited_matches


+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|2<>100670057<>homedepot<>homedepot|1p                            |similar_match_2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg  |1p                       |
|3<>100341666<>homedepot<>homedepot|3p                            |similar_match_3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg  |3p                       |
|6<>100670057<>homedepot<>homedepot|1p_3p                         |similar_match_6<>100670

matches are not missing
+----------------------------------+-------------------------------------------------------------------------------------+--------------------+--------------------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+-------------------+-----------------------+-----------------------+--------------------------+
|base_sku_uuid                     |pair_id                                                                              |base_source_store   |comp_sku_uuid                                     |comp_source_store|seller_type|audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|bungee_audit_status|bungee_auditor         |updated_by             |updated_date              |
+----------------------------------+----------

In [14]:
config = {
    "seller_type" : "1p_over_3p",
    "cardinality" : "1",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
updated_match_suggestion = pruner.prune_matches()
updated_match_suggestion.show(truncate=False, n = 100)

separate_audited_and_unaudited_matches
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|2<>100670057<>homedepot<>homedepot|1p                            |similar_match_2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg  |1p                       |
|3<>100341666<>homedepot<>homedepot|3p                            |similar_match_3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg  |3p                       |
|6<>100670057<>homedepot<>homedepot|1p_3p          

In [15]:
config = {
    "seller_type" : "1p_over_3p",
    "cardinality" : "n",
    "match_type" : "exact"
}

pruner = Match_Pruner(mdw, matches, config)
updated_match_suggestion = pruner.prune_matches()
updated_match_suggestion.show(truncate=False, n = 100)

separate_audited_and_unaudited_matches
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|2<>100670057<>homedepot<>homedepot|1p                            |similar_match_2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg  |1p                       |
|3<>100341666<>homedepot<>homedepot|3p                            |similar_match_3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg  |3p                       |
|6<>100670057<>homedepot<>homedepot|1p_3p          

+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|1<>203370169<>homedepot<>homedepot|1p                            |exact_match_1<>203370169<>homedepot<>homedepot_compYes1pExactSugg1p<>967446<>bcg<>bcg  |1p                       |
|4<>53409<>homedepot<>homedepot    |3p                            |exact_match_4<>53409<>homedepot<>homedepot_compYes3pExactSugg3p<>147664<>bcg<>bcg      |3p                       |
|5<>203370169<>homedepot<>homedepot|1p_3p                         |exact_match_5<>20337016

23/12/13 12:59:52 WARN CacheManager: Asked to cache already cached data.


matches are not missing
+----------------------------------+-------------------------------------------------------------------------------------+--------------------+--------------------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+-------------------+-----------------------+-----------------------+--------------------------+
|base_sku_uuid                     |pair_id                                                                              |base_source_store   |comp_sku_uuid                                     |comp_source_store|seller_type|audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|bungee_audit_status|bungee_auditor         |updated_by             |updated_date              |
+----------------------------------+----------

In [16]:
config = {
    "seller_type" : "1p",
    "cardinality" : "1",
    "match_type" : "similar"
}

pruner = Match_Pruner(mdw, matches, config)
updated_match_suggestion = pruner.prune_matches()
updated_match_suggestion.show(truncate=False, n = 100)

separate_audited_and_unaudited_matches
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|2<>100670057<>homedepot<>homedepot|1p                            |similar_match_2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg  |1p                       |
|3<>100341666<>homedepot<>homedepot|3p                            |similar_match_3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg  |3p                       |
|6<>100670057<>homedepot<>homedepot|1p_3p          

matches are not missing
+-------------------------------------+-------------------------------------------------------------------------------------+--------------------+--------------------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+-------------------+-----------------------+-----------------------+--------------------------+
|base_sku_uuid                        |pair_id                                                                              |base_source_store   |comp_sku_uuid                                     |comp_source_store|seller_type|audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|bungee_audit_status|bungee_auditor         |updated_by             |updated_date              |
+-------------------------------------+-

In [17]:
config = {
    "seller_type" : "1p",
    "cardinality" : "n",
    "match_type" : "similar"
}

pruner = Match_Pruner(mdw, matches, config)
updated_match_suggestion = pruner.prune_matches()
updated_match_suggestion.show(truncate=False, n = 100)

separate_audited_and_unaudited_matches
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|2<>100670057<>homedepot<>homedepot|1p                            |similar_match_2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg  |1p                       |
|3<>100341666<>homedepot<>homedepot|3p                            |similar_match_3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg  |3p                       |
|6<>100670057<>homedepot<>homedepot|1p_3p          

In [18]:
config = {
    "seller_type" : "3p",
    "cardinality" : "1",
    "match_type" : "similar"
}

pruner = Match_Pruner(mdw, matches, config)
updated_match_suggestion = pruner.prune_matches()
updated_match_suggestion.show(truncate=False, n = 100)

separate_audited_and_unaudited_matches


+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|2<>100670057<>homedepot<>homedepot|1p                            |similar_match_2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg  |1p                       |
|3<>100341666<>homedepot<>homedepot|3p                            |similar_match_3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg  |3p                       |
|6<>100670057<>homedepot<>homedepot|1p_3p                         |similar_match_6<>100670

matches are not missing
+-------------------------------------+-------------------------------------------------------------------------------------+--------------------+--------------------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+-------------------+-----------------------+-----------------------+--------------------------+
|base_sku_uuid                        |pair_id                                                                              |base_source_store   |comp_sku_uuid                                     |comp_source_store|seller_type|audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|bungee_audit_status|bungee_auditor         |updated_by             |updated_date              |
+-------------------------------------+-

In [19]:
config = {
    "seller_type" : "3p",
    "cardinality" : "n",
    "match_type" : "similar"
}

pruner = Match_Pruner(mdw, matches, config)
updated_match_suggestion = pruner.prune_matches()
updated_match_suggestion.show(truncate=False, n = 100)

separate_audited_and_unaudited_matches
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|2<>100670057<>homedepot<>homedepot|1p                            |similar_match_2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg  |1p                       |
|3<>100341666<>homedepot<>homedepot|3p                            |similar_match_3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg  |3p                       |
|6<>100670057<>homedepot<>homedepot|1p_3p          

matches are not missing
+-------------------------------------+-------------------------------------------------------------------------------------+--------------------+--------------------------------------------------+-----------------+-----------+------------------------------+---------------------+-------------------------+-------------------+-----------------------+-----------------------+--------------------------+
|base_sku_uuid                        |pair_id                                                                              |base_source_store   |comp_sku_uuid                                     |comp_source_store|seller_type|audited_match_seller_type_list|misc_info            |audited_match_seller_type|bungee_audit_status|bungee_auditor         |updated_by             |updated_date              |
+-------------------------------------+-------------------------------------------------------------------------------------+--------------------+--------------------------

In [20]:
config = {
    "seller_type" : "1p_or_3p",
    "cardinality" : "1",
    "match_type" : "similar"
}

pruner = Match_Pruner(mdw, matches, config)
updated_match_suggestion = pruner.prune_matches()
updated_match_suggestion.show(truncate=False, n = 100)

separate_audited_and_unaudited_matches
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|2<>100670057<>homedepot<>homedepot|1p                            |similar_match_2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg  |1p                       |
|3<>100341666<>homedepot<>homedepot|3p                            |similar_match_3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg  |3p                       |
|6<>100670057<>homedepot<>homedepot|1p_3p          

matches are not missing
+----------------------------------+-------------------------------------------------------------------------------------+--------------------+--------------------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+-------------------+-----------------------+-----------------------+--------------------------+
|base_sku_uuid                     |pair_id                                                                              |base_source_store   |comp_sku_uuid                                     |comp_source_store|seller_type|audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|bungee_audit_status|bungee_auditor         |updated_by             |updated_date              |
+----------------------------------+----------

In [6]:
config = {
    "seller_type" : "1p_or_3p",
    "cardinality" : "n",
    "match_type" : "similar"
}

pruner = Match_Pruner(mdw, matches, config)
updated_match_suggestion = pruner.prune_matches()
updated_match_suggestion.show(truncate=False, n = 100)

separate_audited_and_unaudited_matches
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|2<>100670057<>homedepot<>homedepot|1p                            |similar_match_2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg  |1p                       |
|3<>100341666<>homedepot<>homedepot|3p                            |similar_match_3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg  |3p                       |
|6<>100670057<>homedepot<>homedepot|1p_3p          

matches are not missing
+----------------------------------+-------------------------------------------------------------------------------------+--------------------+--------------------------------------------------+-----------------+-----------+------------------------------+---------+-------------------------+-------------------+--------------+----------+------------+
|base_sku_uuid                     |pair_id                                                                              |base_source_store   |comp_sku_uuid                                     |comp_source_store|seller_type|audited_match_seller_type_list|misc_info|audited_match_seller_type|bungee_audit_status|bungee_auditor|updated_by|updated_date|
+----------------------------------+-------------------------------------------------------------------------------------+--------------------+--------------------------------------------------+-----------------+-----------+------------------------------+---------+---------

In [7]:
config = {
    "seller_type" : "1p_over_3p",
    "cardinality" : "1",
    "match_type" : "similar"
}

pruner = Match_Pruner(mdw, matches, config)
updated_match_suggestion = pruner.prune_matches()
updated_match_suggestion.show(truncate=False, n = 100)

separate_audited_and_unaudited_matches


+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|2<>100670057<>homedepot<>homedepot|1p                            |similar_match_2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg  |1p                       |
|3<>100341666<>homedepot<>homedepot|3p                            |similar_match_3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg  |3p                       |
|6<>100670057<>homedepot<>homedepot|1p_3p                         |similar_match_6<>100670

matches are not missing
+----------------------------------+-------------------------------------------------------------------------------------+--------------------+--------------------------------------------------+-----------------+-----------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+-------------------+-----------------------+-----------------------+--------------------------+
|base_sku_uuid                     |pair_id                                                                              |base_source_store   |comp_sku_uuid                                     |comp_source_store|seller_type|audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|bungee_audit_status|bungee_auditor         |updated_by             |updated_date              |
+----------------------------------+----------

In [8]:
config = {
    "seller_type" : "1p_over_3p",
    "cardinality" : "n",
    "match_type" : "similar"
}

pruner = Match_Pruner(mdw, matches, config)
updated_match_suggestion = pruner.prune_matches()
updated_match_suggestion.show(truncate=False, n = 100)

separate_audited_and_unaudited_matches
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|base_sku_uuid                     |audited_match_seller_type_list|misc_info                                                                              |audited_match_seller_type|
+----------------------------------+------------------------------+---------------------------------------------------------------------------------------+-------------------------+
|2<>100670057<>homedepot<>homedepot|1p                            |similar_match_2<>100670057<>homedepot<>homedepot_compYes1pSimSugg1p<>967493<>bcg<>bcg  |1p                       |
|3<>100341666<>homedepot<>homedepot|3p                            |similar_match_3<>100341666<>homedepot<>homedepot_compYes3pSimSugg3p<>109589<>bcg<>bcg  |3p                       |
|6<>100670057<>homedepot<>homedepot|1p_3p          

23/12/13 13:35:26 WARN CacheManager: Asked to cache already cached data.


matches are not missing
+----------------------------------+-------------------------------------------------------------------------------------+--------------------+--------------------------------------------------+-----------------+-----------+------------------------------+---------+-------------------------+-------------------+--------------+----------+------------+
|base_sku_uuid                     |pair_id                                                                              |base_source_store   |comp_sku_uuid                                     |comp_source_store|seller_type|audited_match_seller_type_list|misc_info|audited_match_seller_type|bungee_audit_status|bungee_auditor|updated_by|updated_date|
+----------------------------------+-------------------------------------------------------------------------------------+--------------------+--------------------------------------------------+-----------------+-----------+------------------------------+---------+---------